In [2]:
#Import libraries
import pyam
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np

<IPython.core.display.Javascript object>

In [3]:
def get_data():
    #get IIASA identification
    iiasa_creds = r"C:\Users\scheifinger\Documents\GitHub\Feasibility_Tool\iiasa_credentials.yml" 
    pyam.iiasa.Connection(creds=iiasa_creds)

    connections = list(pyam.iiasa.Connection(creds=iiasa_creds).valid_connections)
    #other variables: 'Emissions|CO2', 'Primary Energy|Coal', 
    #query for climate scenario data
    df = pyam.read_iiasa(
        name = 'engage_internal',
        creds = iiasa_creds,
        scenario =['T34_550_feas_em',
                   'T34_550_feas_pr',
                   'T34_550_feas_ref',
                   'T34_1000_feas_em',
                   'T34_1000_feas_pr',
                   'T34_1000_feas_ref'],
        variable=["Emissions|CO2", "Policy Cost|Consumption Loss", "Primary Energy|Coal"],
        region=['World']
    )   
    #return data format of df
    return df


In [4]:
data_set = get_data().data

pyam - INFO: Running in a notebook, setting up a basic logging at level INFO
pyam.iiasa - INFO: You are connected as user `scheifinger`
httpx - INFO: HTTP Request: POST https://api.manager.ece.iiasa.ac.at/v1/token/obtain/ "HTTP/1.1 200 OK"
pyam.iiasa - INFO: You are connected as user `scheifinger`
httpx - INFO: HTTP Request: GET https://api.manager.ece.iiasa.ac.at/legacy/applications "HTTP/1.1 200 OK"
httpx - INFO: HTTP Request: POST https://api.manager.ece.iiasa.ac.at/v1/token/obtain/ "HTTP/1.1 200 OK"
httpx - INFO: HTTP Request: GET https://api.manager.ece.iiasa.ac.at/legacy/applications "HTTP/1.1 200 OK"
httpx - INFO: HTTP Request: GET https://api.manager.ece.iiasa.ac.at/legacy/applications/IXSE_ENGAGE_INTERNAL/config "HTTP/1.1 200 OK"
pyam.iiasa - INFO: You are connected to the IXSE_ENGAGE_INTERNAL scenario explorer hosted by IIASA. If you use this data in any published format, please cite the data as provided in the explorer guidelines: https://data.ece.iiasa.ac.at/engage-internal

In [5]:
df = data_set
#from long to wide
df = pd.pivot(data=df, index=['model','scenario', 'region', 'year'], columns = 'variable', values = 'value').reset_index()

In [6]:
#Data wrangling
#get different carbon budgets from scenario name
df['carbon_budget'] = np.where(df['scenario'].str.contains("1000", case=False), "2C", "1.5C")
#get diffeernt scenario narratives from scenario name
df['scenario_narrative'] = np.where(df['scenario'].str.contains("_em", case=True), "EM", np.nan) 
df['scenario_narrative'] = np.where(df['scenario'].str.contains("_ref", case=True), "REF", df['scenario_narrative']) 
df['scenario_narrative'] = np.where(df['scenario'].str.contains("_pr", case=True), "PR", df['scenario_narrative'])


In [7]:
df.head(5)

variable,model,scenario,region,year,Emissions|CO2,Policy Cost|Consumption Loss,Primary Energy|Coal,carbon_budget,scenario_narrative
0,AIM/CGE V2.2,T34_1000_feas_em,World,2010,37007.3548,17.8415,155.4422,2C,EM
1,AIM/CGE V2.2,T34_1000_feas_em,World,2015,39100.9391,18.5841,169.8200,2C,EM
2,AIM/CGE V2.2,T34_1000_feas_em,World,2020,41498.0654,23.8951,172.5914,2C,EM
3,AIM/CGE V2.2,T34_1000_feas_em,World,2025,36040.5284,1715.2574,140.6006,2C,EM
4,AIM/CGE V2.2,T34_1000_feas_em,World,2030,29490.0437,3705.7282,106.2564,2C,EM


In [36]:
#create figure
fig1 = px.line(df,
               x = "year",
               y="Primary Energy|Coal",
               color = "carbon_budget",
               line_dash="model",
               facet_col="scenario_narrative",
               hover_data = {'scenario':True, 'model':True})

fig1

In [135]:
#calculate year that each scenario hit's net zero
netzero_df = df[df["Emissions|CO2"] <= 0].groupby(["model", "scenario", "carbon_budget"])['year'].min().reset_index()
netzero_df
#aggregated cost until 2100
#df["year"] = pd.to_numeric(df['year'])
#df[df["year"] <= 2100].groupby(["model", "scenario"]).agg({"AR5 climate diagnostics|Temperature|Exceedance Probability|1.5 degC|MAGICC6": ['sum']})

,model,scenario,carbon_budget,year
0,AIM/CGE V2.2,T34_1000_feas_em,2C,2060
1,AIM/CGE V2.2,T34_1000_feas_pr,2C,2055
2,AIM/CGE V2.2,T34_1000_feas_ref,2C,2070
3,GEM-E3_V2023,T34_1000_feas_em,2C,2090
4,IMAGE 3.2,T34_1000_feas_em,2C,2070
5,IMAGE 3.2,T34_1000_feas_pr,2C,2070
6,IMAGE 3.2,T34_1000_feas_ref,2C,2070
7,MESSAGEix-GLOBIOM_1.1,T34_1000_feas_em,2C,2080
8,MESSAGEix-GLOBIOM_1.1,T34_1000_feas_pr,2C,2070
9,MESSAGEix-GLOBIOM_1.1,T34_1000_feas_ref,2C,2100


In [134]:
fig2 = px.scatter(
    data_frame=netzero_df,
    x = "year",
    y="carbon_budget"
)

In [22]:

px.line(df.loc[(df['scenario'] == 'T34_1000_feas_em') & (df['model'] == "POLES ENGAGE")],
        x = "year",
        y="Primary Energy|Coal",
        color = "carbon_budget",
        facet_col="scenario_narrative",
        hover_data = {'scenario':True})

In [26]:
df.loc[(df['scenario'] == 'T34_1000_feas_em') & (df['model'] == "REMIND 3.0")]

variable,model,scenario,region,year,Emissions|CO2,Policy Cost|Consumption Loss,Primary Energy|Coal,carbon_budget,scenario_narrative
258,REMIND 3.0,T34_1000_feas_em,World,2005,36076.658422,0.000000,141.519484,2C,EM
259,REMIND 3.0,T34_1000_feas_em,World,2010,39344.568089,0.000000,157.164604,2C,EM
260,REMIND 3.0,T34_1000_feas_em,World,2015,42044.900833,0.000000,167.482203,2C,EM
261,REMIND 3.0,T34_1000_feas_em,World,2020,43169.892441,0.000000,165.331930,2C,EM
262,REMIND 3.0,T34_1000_feas_em,World,2025,39424.985445,1290.026481,144.765797,2C,EM
263,REMIND 3.0,T34_1000_feas_em,World,2030,30030.384835,2805.458360,90.005479,2C,EM
264,REMIND 3.0,T34_1000_feas_em,World,2035,23516.017394,3469.401994,59.804043,2C,EM
265,REMIND 3.0,T34_1000_feas_em,World,2040,17357.489268,4201.939569,30.724913,2C,EM
266,REMIND 3.0,T34_1000_feas_em,World,2045,13002.879928,5316.244260,17.145052,2C,EM
267,REMIND 3.0,T34_1000_feas_em,World,2050,10165.260117,7195.300250,11.023366,2C,EM
